In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix, lil_matrix
from scipy import sparse
from abc import ABCMeta, abstractmethod
from scipy.stats import randint as sp_randint
import random

In [2]:
!pip install -q xlrd
!git clone https://github.com/mansisinha/taddhita.git
  

fatal: destination path 'taddhita' already exists and is not an empty directory.


In [3]:
!ls taddhita/newPairs.txt

taddhita/newPairs.txt


In [4]:
import pandas as pd
url='https://raw.githubusercontent.com/mansisinha/taddhita/master/allPairs.csv'

df=pd.read_csv(url)
df.drop(['affix','word','sense','meaning','Source Word Count','Derived word count','word_y'],axis=1,inplace=True)
#df['n']=1
df
#df.groupby('Affix').count()

,Affix,Source Word,Derived Word
0,ly,selfish,selfishly
1,ly,virulent,virulently
2,ly,gentle,gently
3,ly,harmonious,harmoniously
4,ly,guarded,guardedly
5,ly,gushing,gushingly
6,ly,habitual,habitually
7,ly,handy,handily
8,ly,haphazard,haphazardly
9,ly,happy,happily


In [5]:
# Iterative longest contiguous sequence. No one character matchings
def lcs(s1,s2):
    longest = ""
    i = 0
    for x in s1:
        if re.search(x, s2):
          s = x
          while re.search(s, s2):
            if len(s)>len(longest):
                longest = s
            if i+len(s) == len(s1):
                break
            s = s1[i:i+len(s)+1]
        i += 1
    return longest

def iterLCS(pdf):
    posCounter = 1
    sw1Orig = pdf['Source Word']
    sw2Orig = pdf['Derived Word']
    sw1 = pdf['Source Word']
    sw2 = pdf['Derived Word']
    longDict = dict()
    prevSInd = 0
    prevTInd = 0
    rejLength =0
    
    while True:
        tempVal = lcs(sw1Orig,sw2Orig)
        if len(tempVal)  <= 1:
            break
        
        tempSInd =  pdf['Source Word'].find(tempVal) + 1
        tempTInd =  pdf['Derived Word'].find(tempVal) + 1
        if (tempSInd - prevSInd)*(tempTInd - prevTInd) < 0:
            print('here',tempVal,pdf['Source Word'],pdf['Derived Word'])
            sw1Orig = sw1Orig.replace(tempVal,'~~',1)
            sw2Orig = sw2Orig.replace(tempVal,'^^',1)
            rejLength += len(tempVal)
        else:
            longDict[tempSInd] = tempVal+'#'+str(posCounter)
            sw1 = sw1.replace(tempVal,'#'+str(posCounter)+'#',1)
            sw2 = sw2.replace(tempVal,'!'+str(posCounter)+'!',1)
            sw1Orig = sw1Orig.replace(tempVal,'~'+str(posCounter)+'~',1)
            sw2Orig = sw2Orig.replace(tempVal,'^'+str(posCounter)+'^',1)
            prevSInd = tempSInd + 1
            prevTInd = tempTInd + 1
            posCounter += 1
    
    #return [longList,[item for item in sw1.split('#') if len(item) > 0],[item for item in sw2.split('!') if len(item) > 0]]
    pdf['common'] = longDict
    pdf['deleted'] = [item for item in sw1.split('#') if len(item) > 0 and item.isdigit()!=True ]
    pdf['sourceSkeleton'] = sw1
    pdf['targetSkeleton'] = sw2
    

    jointWord = ''
    sw1Split = sw1.split('#')
    sw2Split = sw2.split('!')
    
    sw3 = [nummi for nummi in sw1Split if nummi.isdigit() == True]
    sw4 = [nummi for nummi in sw2Split if nummi.isdigit() == True]
    
    if sw3!=sw4:
            print('ordermatchilla',sw3,sw4)
    

    for key in sorted(longDict.keys()):
        nowN = longDict[key].split('#')[1]
        try:
            
            while sw1Split[0] != nowN:
            
                tingi = sw1Split.pop(0)
                if len(tingi) > 0:
                    
                    jointWord += 'DEL('
                    for cahra in tingi:
                        jointWord += cahra
                    jointWord += ')'
        except IndexError:
            print("SW1",pdf['Source Word'])
                
        try:
            while sw2Split[0] != nowN:
                tingi = sw2Split.pop(0)
                if len(tingi) > 0:
                    jointWord += 'INS('
                    for cahra in tingi:
                        jointWord += cahra
                    jointWord += ')'
        except IndexError:
            print("SW2",pdf['source'])
            
            
        try:    
            if  sw1Split[0] == nowN and  sw2Split[0] == nowN:
                jointWord = jointWord + longDict[key].split('#')[0]
                sw1Split.pop(0)
                sw2Split.pop(0)
            else:
                print('pani pali not expected')
        except IndexError:
            print("common",pdf['Source Word'])
                
    while len(sw1Split) > 0:
        tingi = sw1Split.pop(0)
        if len(tingi) > 0:
            jointWord += 'DEL('
            for cahra in tingi:
                jointWord += cahra
            jointWord += ')'
            
    while len(sw2Split) > 0:
        
        tingi = sw2Split.pop(0)
        if len(tingi) > 0:
            jointWord += 'INS('
            for cahra in tingi:
                jointWord += cahra
            jointWord += ')'

    
    if len(pdf['deleted']) == 0:
        pdf['deleted'] = ['ϵ']
    
    
    pdf['added'] = [item for item in sw2.split('!') if len(item) > 0 and item.isdigit()!=True]
    
    if len(pdf['added']) == 0:
        pdf['added'] = ['ϵ']
        
    pdf['aligned'] = jointWord
    
    return pdf
    
#iterLCS(ppd.iloc[3865,:])

In [12]:
import re
df12=df.sample(frac=1)
ddf = df12.apply(iterLCS, axis=1)

In [35]:
ddf

,Affix,Source Word,Derived Word,common,deleted,sourceSkeleton,targetSkeleton,added,aligned
3968,ness,glib,glibness,{1: 'glib#1'},[ϵ],#1#,!1!ness,[ness],glibINS(ness)
7043,ment,recruit,recruitment,{1: 'recruit#1'},[ϵ],#1#,!1!ment,[ment],recruitINS(ment)
5447,let,script,scriptlet,{1: 'script#1'},[ϵ],#1#,!1!let,[let],scriptINS(let)
4767,or,impropriate,impropriator,{1: 'impropriat#1'},[e],#1#e,!1!or,[or],impropriatDEL(e)INS(or)
2872,ist,apiology,apiologist,{1: 'apiolog#1'},[y],#1#y,!1!ist,[ist],apiologDEL(y)INS(ist)
4860,ee,pierce,piercee,{1: 'pierce#1'},[ϵ],#1#,!1!e,[e],pierceINS(e)
3740,ness,morbid,morbidness,{1: 'morbid#1'},[ϵ],#1#,!1!ness,[ness],morbidINS(ness)
4216,ic,taxonomy,taxonomic,{1: 'taxonom#1'},[y],#1#y,!1!ic,[ic],taxonomDEL(y)INS(ic)
3187,ic,chalcedony,chalcedonic,{1: 'chalcedon#1'},[y],#1#y,!1!ic,[ic],chalcedonDEL(y)INS(ic)
1280,ly,clerk,clerkly,{1: 'clerk#1'},[ϵ],#1#,!1!ly,[ly],clerkINS(ly)


In [331]:
pattern =set()
for index, row in ddf.iterrows():
    pattern.add((row['sourceSkeleton'],row['targetSkeleton']))

In [44]:
print(pattern)

{('#1#m', '!1!t'), ('#1#en', '!1!ic'), ('#1#', '!1!t'), ('#1#e', '!1!ment'), ('bo#1#re', 'b!1!ger'), ('#1#e', '!1!ing'), ('#1#', '!1!page'), ('#1#r', '!1!ment'), ('#1#', '!1!ize'), ('#1#er', '!1!ress'), ('#1#ne#2#', '!1!!2!'), ('#1#e', '!1!ic'), ('#1#or', '!1!ress'), ('#1#', '!1!ty'), ('#1#e', '!1!ical'), ('bou#1#', 'but!1!'), ('#1#e', '!1!ism'), ('#1#e', '!1!t'), ('g#1#', 'gu!1!'), ('#1#', '!1!ive'), ('#1#', '!1!m'), ('#1##2#', '!1!t!2!'), ('#1#is', '!1!al'), ('#1#ish', '!1!'), ('#1##2#', '!1!l!2!'), ('#1#', '!1!tic'), ('#1#ician', '!1!o'), ('#1#', '!1!ous'), ('#1#', '!1!ie'), ('ch#1#', 'j!1!'), ('#1#ic', '!1!oid'), ('#1#e', '!1!al'), ('#1#tic', '!1!'), ('#1#e', '!1!ification'), ('feig#1#', 'faw!1!ly'), ('#1#e#2#r', '!1!!2!ss'), ('#1#y', '!1!ian'), ('#1#y#2#', '!1!i!2!'), ('xer#1#', 'mes!1!'), ('#1#us', '!1!ise'), ('#1#', '!1!zer'), ('#1#', '!1!den'), ('#1#', '!1!do'), ('#1#', '!1!gle'), ('#1#', '!1!ness'), ('#1#', '!1!oid'), ('#1#e', '!1!iferous'), ('#1#on', '!1!ic'), ('#1#e', '!1!is

In [32]:
import io,os
vocab = []
gloveDir='/home/bt1/17CS10037/glove'
with io.open(os.path.join(gloveDir, 'glove.840B.300d.txt'), encoding="utf8") as f:
    for line in f:
        values = line.split()
        vocab.append(values[0])
print('Found %s word vectors.' % len(vocab))

Found 2196017 word vectors.


In [497]:
def lcs1(X, Y, m, n): 
    L = [[0 for x in range(n+1)] for x in range(m+1)]   
    for i in range(m+1): 
        for j in range(n+1): 
            if i == 0 or j == 0: 
                L[i][j] = 0
            elif X[i-1] == Y[j-1]: 
                L[i][j] = L[i-1][j-1] + 1
            else: 
                L[i][j] = max(L[i-1][j], L[i][j-1])  
    index = L[m][n]  
    lcs = [""] * (index+1) 
    lcs[index] = "" 
    i = m 
    j = n 
    left=[]
    flag=1
    flag1=0
    right=[]
    while i > 0 and j > 0: 
        #print(X[i-1],' ',Y[j-1])
        if X[i-1] == Y[j-1]: 
            lcs[index-1] = X[i-1] 
            i-=1
            right.extend(Y[j-1])
            j-=1
            index-=1
            if i==0 :
                #print(',')
                left.extend(',')
            flag=0
            flag1=0
        elif L[i-1][j] > L[i][j-1]:
            if flag==0:
                left.extend(',')
                flag=1
            if flag1==0:
                right.extend(',')
                flag1=1
            i-=1
        else:
            if flag==0:
                #print(',')
                left.extend(',')
                flag=1
            if flag1==0:
                right.extend(',')
                flag1=1
            left.extend(Y[j-1])
            j-=1
    while i > 0:
        i-=1
    while j > 0:
        left.extend(Y[j-1])
        j-=1
    string="".join(left)
    string1=string[::-1]
    string5="".join(right)
    string6=string5[::-1]
#     print(string6)
    string2=[]
    i=0
    while i<(len(X)):
        if X[i]=='!':
            if i!=0:
                string2.extend(','+X[i]+X[i+1]+X[i+2])
            else:
                string2.extend(X[i]+X[i+1]+X[i+2])
            i+=3
        else:
            i+=1
    if(X[i-1])!='!':
        string2.extend(',')
    string3="".join(string2)
    return "".join(lcs),string1,string3,string6

In [498]:
str1='!2!page!4!ion'
str2='agetyuion'
print(lcs1(str1,str2,len(str1),len(str2)))
  

('ageion', 'tyu,', '!2!,!4!,', 'age,ion')


,Affix,Source Word,Derived Word,common,deleted,sourceSkeleton,targetSkeleton,added,aligned
0,ly,breakbest,unbreakhellobestable,"{1: 'break#1', 6: 'best#2'}",[ϵ],#1##2#,un!1!hello!2!able,"[un, hello, able]",INS(un)breakINS(hello)bestINS(able)
1,ly,virulent,virulently,{1: 'virulent#1'},[ϵ],#1#,!1!ly,[ly],virulentINS(ly)
2,ly,gentle,gently,{1: 'gentl#1'},[e],#1#e,!1!y,[y],gentlDEL(e)INS(y)
3,ly,harmonious,harmoniously,{1: 'harmonious#1'},[ϵ],#1#,!1!ly,[ly],harmoniousINS(ly)
4,ly,guarded,guardedly,{1: 'guarded#1'},[ϵ],#1#,!1!ly,[ly],guardedINS(ly)
5,ly,gushing,gushingly,{1: 'gushing#1'},[ϵ],#1#,!1!ly,[ly],gushingINS(ly)
6,ly,habitual,habitually,{1: 'habitual#1'},[ϵ],#1#,!1!ly,[ly],habitualINS(ly)
7,ly,handy,handily,{1: 'hand#1'},[y],#1#y,!1!ily,[ily],handDEL(y)INS(ily)
8,ly,haphazard,haphazardly,{1: 'haphazard#1'},[ϵ],#1#,!1!ly,[ly],haphazardINS(ly)
9,ly,happy,happily,{1: 'happ#1'},[y],#1#y,!1!ily,[ily],happDEL(y)INS(ily)


In [604]:
def check(x,y):
    z,w,u,v=lcs1(x,y,len(x),len(y))
    u=u.split(',')
    w=w.split(',')
#     print(u)
#     print(w)
    if(len(w)!=len(u)):
        return -1
    t=[]
    t.append([])
    i=0
    j=0
    while i <len(x):
        if x[i]=='!':
            if(i!=0):
                t[j]="".join(t[j])
            i+=3
            j+=1
            t.append([])
        else:
            t[j].extend(x[i])
            i+=1
    t[j]="".join(t[j])
#     print(t)
    v=v.split(',')
#     print(v)
    i=0
    while i <len(t):
        try:
            if len(t[i])==0 and v[i]!='':
                return -1
            elif  len(t[i])!=0 and v[i]=='':
                return -1
            elif  len(t[i])==0 and v[i]=='':
                i+=1
                continue
            elif t[i]!=v[i]:
                return -1
        except IndexError:
            return -1
        i+=1
    
    i=0
    p=[]
    while i<len(w):
        if w[i]=='' and u[i]!='':
            return -1
        elif w[i]!='' and u[i]=='':
            return -1
        if w[i]=='' and u[i]=='':
            i+=1
            continue
        p.append(w[i])
        i+=1
    
    return p

In [605]:
check('xy!2!ty!3!ui','xybaatyerui')

['baa', 'er']

In [606]:
def add(X,Y):
    i=0 
    j=0
    while i<(len(X)):
        if X[i]=='#':
            i+=3
            j+=1
        else:
            i+=1
    string2=[]
    for i in range(j+1) :
        string2.append([])
#     print("add wala")
#     print(string2)
    i=j=0
    while i<(len(X)):
        if X[i]=='#':
            j+=1
            i+=3
        else:
            string2[j].extend(X[i])
            i+=1
#     print(j)
#     print(len(Y))
    if (j)!=(len(Y)):
        return -1
#     print(string2)
    z=[]
    i=0
    while i < j:
        z.extend(string2[i])
        z.extend(Y[i])
        i+=1
    z.extend(string2[i])
    return "".join(z)
    

In [607]:
def transform(x,y):
    X=x[1]
    Y=check(X,y)
    if(Y==-1):
        return -1
#     print(x[0])
#     print(Y)
    z=add(x[0],Y)
    return z
    

In [608]:
# a=pattern.pop()
# pattern.add(a)
# print(a)
a=('in#1##2#', 'bo!1!in!2!')
pat=('#1#', '!1!page')
string='package'
transform(pat,string)

-1

In [611]:
print(pattern)

{('#1#m', '!1!t'), ('#1#en', '!1!ic'), ('#1#', '!1!t'), ('#1#e', '!1!ment'), ('bo#1#re', 'b!1!ger'), ('#1#e', '!1!ing'), ('#1#', '!1!page'), ('#1#r', '!1!ment'), ('#1#', '!1!ize'), ('#1#er', '!1!ress'), ('#1#ne#2#', '!1!!2!'), ('#1#e', '!1!ic'), ('#1#or', '!1!ress'), ('#1#', '!1!ty'), ('#1#e', '!1!ical'), ('bou#1#', 'but!1!'), ('#1#e', '!1!ism'), ('#1#e', '!1!t'), ('g#1#', 'gu!1!'), ('#1#', '!1!ive'), ('#1#', '!1!m'), ('#1##2#', '!1!t!2!'), ('#1#is', '!1!al'), ('#1#ish', '!1!'), ('#1##2#', '!1!l!2!'), ('#1#', '!1!tic'), ('#1#ician', '!1!o'), ('#1#', '!1!ous'), ('#1#', '!1!ie'), ('ch#1#', 'j!1!'), ('#1#ic', '!1!oid'), ('#1#e', '!1!al'), ('#1#tic', '!1!'), ('#1#e', '!1!ification'), ('feig#1#', 'faw!1!ly'), ('#1#e#2#r', '!1!!2!ss'), ('#1#y', '!1!ian'), ('#1#y#2#', '!1!i!2!'), ('xer#1#', 'mes!1!'), ('#1#us', '!1!ise'), ('#1#', '!1!zer'), ('#1#', '!1!den'), ('#1#', '!1!do'), ('#1#', '!1!gle'), ('#1#', '!1!ness'), ('#1#', '!1!oid'), ('#1#e', '!1!iferous'), ('#1#on', '!1!ic'), ('#1#e', '!1!is

In [610]:
# i=0
# for pat in pattern:
    pat=('#1#ne#2#','!1!!2!')
    i=0
    print('\n\n')
    print(pat[0])
    print(pat[1])
    print('\n\n')
    for der in vocab:
        if(i==5):
            break
        a=transform(pat,der)
        if a==-1:
            continue
        for sou in vocab:
            if sou==a:
                print(sou,end='\t')
                print(der)
                i+=1
                break




#1#ne#2#
!1!!2!



